<a href="https://colab.research.google.com/github/divassya/BigDataAnalysis/blob/main/AssiyaKaratay_Assignment_2_Grades_PiEstimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Assignment 2
MET CS777 Big Data Analytics 


Faculty - Farshid Alizadeh-Shabdiz, PhD, MBA

Student - Assiya Karatay U95161396 karatay@bu.edu 857-294-7028


### Problem 1 (10 points)

What are the differences between following RDD operations from functionality’s perspective, and also computation and cost of operation.
Assign a level of computation complexity from level 1 (less costly) to level 3 (most costly) to each one. 

P.S. I added documentation for my future use.

#### 1.	aggregateByKey() - level 2 
- From documentation: Aggregate the values of each key, using given combine functions and a neutral “zero value”. This function can return a different result type, U, than the type of the values in this RDD, V. 

This is the same as reduceByKey with addition of an initial value. This method is more efficient than groupByKey operation.


#### 2.	reduceByKey() - level 2 
- from documentation """
        Merge the values for each key using an associative and commutative reduce function.

        This will also perform the merging locally on each mapper before
        sending results to a reducer, similarly to a "combiner" in MapReduce.

        Output will be partitioned with `numPartitions` partitions, or
        the default parallelism level if `numPartitions` is not specified.
        Default partitioner is hash-partition.

        Examples
        --------
        >>> from operator import add
        >>> rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
        >>> sorted(rdd.reduceByKey(add).collect())
        [('a', 2), ('b', 1)]
        """
What I understand is that there are partitions. Then data is collected like shuffling. There is only one output for one key at each partition to send over the network. This method is more efficient than groupByKey operation.

#### 3.	groupByKey() - level 3 - from documentation: Group the values for each key in the RDD into a single sequence. Hash-partitions the resulting RDD with numPartitions partitions.

Note:If we are grouping in order to perform an aggregation (such as a sum or average) over each key, using reduceByKey or aggregateByKey will provide much better performance

 What I understand is that groupByKey's memory usage is huge, so it can cause out of disk problems as data is sent over the network and collected on the reduced workers. This function is a little bit inefficient, but it can be used when necessary. 

#### 4.	combineByKey() - level 1 
- From documentation  """
        Generic function to combine the elements for each key using a custom
        set of aggregation functions.

        Turns an RDD[(K, V)] into a result of type RDD[(K, C)], for a "combined
        type" C.

        Users provide three functions:

            - `createCombiner`, which turns a V into a C (e.g., creates
              a one-element list)
            - `mergeValue`, to merge a V into a C (e.g., adds it to the end of
              a list)
            - `mergeCombiners`, to combine two C's into a single one (e.g., merges
              the lists)

        To avoid memory allocation, both mergeValue and mergeCombiners are allowed to
        modify and return their first argument instead of creating a new C.

        In addition, users can control the partitioning of the output RDD.

        Notes
        -----
        V and C can be different -- for example, one might group an RDD of type
            (Int, Int) into an RDD of type (Int, List[Int]).

        Examples
        --------
        >>> x = sc.parallelize([("a", 1), ("b", 1), ("a", 2)])
        >>> def to_list(a):
        ...     return [a]
        ...
        >>> def append(a, b):
        ...     a.append(b)
        ...     return a
        ...
        >>> def extend(a, b):
        ...     a.extend(b)
        ...     return a
        ...
        >>> sorted(x.combineByKey(to_list, append, extend).collect())
        [('a', [1, 2]), ('b', [1])]
        """

This is more complicated, but provides more room for wrangling inputs. Three parameters are passed as input such as an initial value, merging and combine function. I does not need to pass constants always, a function can be passed too that will return a new value. This is the least costly operation among this set. 

### Problem 2 (5 points)
Name at least four differences between Spark and Hadoop MapReduce?
• Speed 
- Batch Processing is not sutable for some business cases
- Spark builds its query computations as a directed acyclic graph (DAG); its DAG scheduler and query optimizer construct an efficient computational graph that can usually be decomposed into tasks that are executed in parallel across workers on the cluster. Its physical execution engine, Tungsten, uses whole-stage code generation to generate compact code for execution (from slides). This means that Spark has better speed on processing than MapReduce. 
• Ease of use
- MapReduce needs Java which is not as user-friendly as other languages
- Spark has simple logical data structure, RDD at the core
• Modularity
- Spark supports most programming languages: Scala, Java, Python, SQL,and R and Components like: Spark SQL, Spark ML, Spark Streaming, and Graphx
• Extensibility
- Spark decouples storage and compute, unlike Hadoop.
- Read and write from many sources like Apache Kafka, Kinesis, Azure storage, AWS S3, etc.

### Problem 3 (5 points)

How does Spark run an application and what are the driver layer functionalities? Explain from the Spark architecture’s perspective. 
 - The driver layer two main functionalities are converting user programs into tasks and planning the execution of tasks by executors. In details, 
* it runs the main() function of the application 
* it schedules the job execution.
* Spark Context and RDDs are created there, and their metadata is saved there
* It translates the RDD’s into the execution graph and splits the graph into multiple stages.
* It converts the application into smaller execution tasks. Tasks are then executed by the executors. After the task has been completed, all the executors submit their results to the Driver.
* It exposes the information about all the process through a Web UI

### Problem 4 (5 points)

What are the differences of running on multi-core computer versus running on multi worker/executor environment? 
List pros and cons of each one.

* Multi-core computer means a computer with more than one processing units. It can be used for multi-threading of the task with shared memory like hard drive, RAM, and cache. 

* Multi worker environment shares nothing in terms of data, processes,connection between machines. Multi worker environment can have (and usually is better to have) multi-core computers. It is much more efficient than one multi-core computer when the data is large (gigabytes, petabytes).
 
### Problem 5 (10 points)

Why RDD is immutable? Was this a mistake in design of RDDs or has some advantage?
RRD's immutability means our code does not change when we copy objects in RDD. It has many benefits:
1. easy to parallelize: When we use  parts of the data in workers, we are certain that the data at the source is not modified. 
2. This prevents from a set of potential problems due to updates from multiple threads at once. Immutable data is definitely safe to share across processes.
3. the RDD's parts can be recreated at any time. This makes caching, sharing and replication easy.
4. gaining the fault tolerance and correctness with no developer effort is worth spending memory and CPU on, since the latter are cheap.

### Problem 6 (10 points)

Spark transformation is divided to narrow transformation and wide transformation. Referring to spark documentation, explain the differences.

* Narrow transformations are processed on a single partition with no data movement between partitions .Examples: map(), mapPartition(), flatMap(), filter(), union() 
* Wide transformations are processed on many partition swith data movements between partitions. It is expensive due to shuffling. Examples:groupByKey(), aggregateByKey(), aggregate(), join(), repartition() 

### Problem 7 (15 points)

List 10 spark transformation operation with one line of example
1. rdd.map(x=>x+2) on RDD {1, 2, 3, 4, 5} applied to every element returns only one element (3, 4, 5, 6, 7).
2. rdd.flatMap(lines => lines.split(” “)) splits each input string into words when space occurs. flatMap() can return a list of elements.
3. filter() function returns a new RDD, containing only the elements that meet a condition. rdd.filter(value => value%2==0) returns even numbers.
4. union() function gives the elements of both the same type RDD in new RDD. RDD1 = (Spark, Hadoop, Flink) and RDD2 = (Big data, Spark, Flink), so the resultant rdd1.union(rdd2) will be (Spark, Spark, Hadoop, Flink, Flink, Big data).
5. intersection() gives the common element in both RDDs (Spark, Spark, Hadoop, Flink) and (Big data, Spark, Flink), so the resultant rdd1.intersection(rdd2) will have elements (spark, flink).
6. rdd.distinct() is helpful to remove duplicate data  and gives elements (Spark, Hadoop, Flink) from (Spark, Spark, Hadoop, Flink).
7. groupByKey() shuffles the data according to the key value in another RDD: spark.sparkContext.parallelize(Array((‘k’,5),(‘s’,3),(‘s’,4),(‘p’,7),(‘p’,5),(‘t’,8),(‘k’,6)),3) group = data.groupByKey().collect()
8. sortByKey() sorts the data according to the key in another RDD: data = spark.sparkContext.parallelize(Seq((“maths”,52), (“english”,75), (“science”,82), (“computer”,65), (“maths”,85))) sorted = data.sortByKey() sorts the data RDD into Ascending order of the Key(String).
9. join() operation combines two data sets on the basis of the key.
data = spark.sparkContext.parallelize(Array((‘A’,1),(‘b’,2),(‘c’,3)))
data2 =spark.sparkContext.parallelize(Array((‘A’,4),(‘A’,6),(‘b’,7),(‘c’,3),(‘c’,8)))
result = data.join(data2)
10. coalesce() avoids full shuffling of data and uses existing partition so that less data is shuffled. Using this we can cut the number of the partition, e.g. only two nodes out of 4. 
rdd1 = spark.sparkContext.parallelize(Array(“jan”,”feb”,”mar”,”april”,”may”,”jun”),3)
val result = rdd1.coalesce(2)

List 5 spark action operation with one line example.
1. rdd.count() returns the number of elements 8 from rdd={1, 2, 2, 3, 4, 5, 5, 6}
    countByValue() returns how many times each element occur {(1,1), (2,2), (3,1), (4,1), (5,2), (6,1)} in RDD = {1, 2, 2, 3, 4, 5, 5, 6}   
2. result.collect() returns our entire RDDs content to driver program. The application of collect() is unit testing where the entire RDD is expected to fit in memory. It has a constraint that all the data should fit in the machine, and copies to the driver.
3.  result.take(n) returns n number of elements as a biased collection from RDD.
4. fold() is like reduce(). Besides, it takes “zero value” as input, which is used for the initial call on each partition. But, the condition with zero value is that it should be the identity element of that operation. The key difference between fold() and reduce() is that, reduce() throws an exception for empty collection, but fold() is defined for empty collection. For example, zero is an identity for addition; one is identity element for multiplication. 
additionalMarks = (“extra”, 4)
sum = rdd1.fold(additionalMarks){ (acc, marks) => val add = acc._2 + marks._2
Note – additionalMarks is an initial value. This value will be added to the int value of each record in the source RDD.
5. foreach() applies operation on each element of RDD, but it does not return value to the driver. For example, inserting a record into the database.
data = spark.sparkContext.parallelize(Array((‘k’,5),(‘s’,3),(‘s’,4),(‘p’,7),(‘p’,5),(‘t’,8),(‘k’,6)),3)
group = data.groupByKey().collect()
group.foreach(println) # run a function (println) on each element of the dataset group.

### Problem 8 (20 points)

We have a data which consists of following columns
-	Row number
-	First name of student
-	Last name of student
-	Course number
-	Grade 

Write an efficient Spark code to calculate
1.	Min grade of each student
2.	Max grade of each student
3.	GPA 
4.	Number of courses taken



In [8]:
# Spark installation on Colab
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# !tar xf spark-3.0.1-bin-hadoop3.2.tgz
# !pip install -q findspark
# !rm -rf spark-3.0.1-bin-hadoop3.2.tgz

!pip install --ignore-installed -q pyspark==3.1.2 

In [9]:
# Set JAVA_HOME and SPARK_HOME
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "spark-3.0.1-bin-hadoop3.2"

# import findspark
# findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME


import sys
import requests
from operator import add

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [10]:
# sample data
data = [(0,0,'James','Smith','CS767',93),
  (1,1,'Michael','Rose','MA582',85),
  (2,2,'Robert','Williams','CS767',90),
  (3,3,'Maria','Jones','CS777',75),
  (4,4,'Jen','Brown','CS779',80),
  (5,0,'James','Smith','CS555',80),
  (6,1,'Michael','Rose','CS555',100)
]

columns = ['rownumber', 'id', "firstname", "lastname", "coursenumber", "grade"]
df = spark.createDataFrame(data=data, schema = columns)

In [11]:
df.show()

+---------+---+---------+--------+------------+-----+
|rownumber| id|firstname|lastname|coursenumber|grade|
+---------+---+---------+--------+------------+-----+
|        0|  0|    James|   Smith|       CS767|   93|
|        1|  1|  Michael|    Rose|       MA582|   85|
|        2|  2|   Robert|Williams|       CS767|   90|
|        3|  3|    Maria|   Jones|       CS777|   75|
|        4|  4|      Jen|   Brown|       CS779|   80|
|        5|  0|    James|   Smith|       CS555|   80|
|        6|  1|  Michael|    Rose|       CS555|  100|
+---------+---+---------+--------+------------+-----+



In [12]:
minGrade = df.groupBy("id").agg({'grade':'min'})
maxGrade = df.groupBy("id").agg({'grade':'max'})
GPA = df.groupBy("id").agg({'grade':'mean'})
numberOfCourses = df.groupBy("id").agg({'coursenumber':'count'})

In [13]:
minGrade.show()
maxGrade.show()
GPA.show()
numberOfCourses.show()

+---+----------+
| id|min(grade)|
+---+----------+
|  0|        80|
|  1|        85|
|  3|        75|
|  2|        90|
|  4|        80|
+---+----------+

+---+----------+
| id|max(grade)|
+---+----------+
|  0|        93|
|  1|       100|
|  3|        75|
|  2|        90|
|  4|        80|
+---+----------+

+---+----------+
| id|avg(grade)|
+---+----------+
|  0|      86.5|
|  1|      92.5|
|  3|      75.0|
|  2|      90.0|
|  4|      80.0|
+---+----------+

+---+-------------------+
| id|count(coursenumber)|
+---+-------------------+
|  0|                  2|
|  1|                  2|
|  3|                  1|
|  2|                  1|
|  4|                  1|
+---+-------------------+




### Problem 9 (20 points)

Estimation area of a circle:

Use Spark to estimate area of the unit circle by "throwing darts" at the circle. 
Assume you don’t know how to calculate area of a circle in a closed form, but you know how to calculate area of a square. You throw random darts/points in the 2 by 2 square ((-1, -1) to (1,1)) and count how many falls in the unit circle, a circle with radius of one. The fraction can be used to estimate of the area of the unit circle.

Idea of solution:
Area of the Unit Circle C= pi*r**2 = pi
Area of the square S = 2**2 = 4

The fraction of Circle Area by Square Area 
C/S= Pi/4

C = Pi*S/4 = Pi*4/4 = Pi

0. Parallelize the data of size n
1. throw random darts/points in the 2 by 2 square ((-1, -1) to (1,1))  = get coordinates of points from the Uniform Distribution between -1 and 1
2. return true if the point is in a circle using the inequality x*x + y*y < 1, otherwise return False.
3. count how many falls in the unit circle = get the proportion of points inside of the circle 


In [14]:
from random import uniform
from operator import add

n = 100000 

def isInCircle(_):
    x, y = uniform(-1,1), uniform(-1,1)
    return (x * x + y * y < 1)

rdd = sc.parallelize(range(1, n + 1))
count = rdd.map(isInCircle).reduce(add)
print("Estimation area of the unit circle is " + str( 4*count / n))

Estimation area of the unit circle is 3.13536


### Notes
This was great introduction to Spark and Hadoop. Initially when I wrote answers to question 1-8, it was easier to feel that I understand what I am doing. In fact, when I got errors compiling the solution for Problem 8, it was tough. Then I got adapted and realized that pyspark is so close to Python. The solution of estimating Pi exists so many years, the question was about how to implement it using map reduce ideas. It was beneficial to solve this assignment in terms of both theory and practice.   